# Part 5

Это последнее занятие в курсе, и сегодня мы наконец поставим точку. Мы создадим своего бота!

In [ ]:
!pip install pyTelegramBotAPI
!pip install transformers

## Модель

За основу мы возьмем модель `RuGPT3` - модель от Сбера, которая может генерировать сообщения.

В теории, можно использовать и другие модели - их можно найти на [huggingface](https://huggingface.co/models) - важно правильно воспользоваться фильтрами, а то велика вероятность скачать что-то не то)

Тут потребуется включенная в колабе GPU - для этого тыкаем сверху `Среда выполнения (Runtime)` → `Сменить среду выполнения (change runtime)` → выбираем GPU → нажимаем `Сохранить`

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2").cuda()

## Функция предсказания

Эта функция возвращает сгенерированный моделью текст

In [ ]:
# beam search

def generate_text(text, max_letters):
  ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0).cuda()
  generated_ids = model.generate(
                          input_ids=ids,  # input
                          max_length=max_letters,  # default 20
                          min_length=0,  # default 0
                          do_sample=True,  # don't use greedy decoding
                          early_stopping=False,  # search is stopped when at least num_beams sentences finished
                          temperature=2.45,  # default 1.0
                          top_k=45,  # default 50
                          top_p=0.7,  # default 1.0
                          repetition_penalty=2.0,  # rep. penalty
                          num_beams=6,
                          num_return_sequences=1, #  num ind. computed returned sequences
                          bos_token_id=tokenizer.bos_token_id
                          )
  result = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  return result

In [ ]:
print(generate_text('Однажды мы с друзьями пришли на пару по NLP.', 200).replace('<s>', ''), '\n')

Прикольно, не правда ли?

## Бот

Для кого-то это будет открытием, но на питоне можно писать телеграм ботов! Причем делается это довольно несложно. Если появится желание создать бота с нуля - почитайте [несложную статейку в хабре](https://habr.com/ru/post/580408/)

Если вы хотите его запустить - придется его создать :)

Для этого делаем следущее:



1.   Переходим на https://t.me/BotFather
2.   Пишем там `/newbot`
3.   Пишем для него имя (оно будет отображаться)
4.   Пишем для него ник (он должен заканчиваться на `bot`)
5.   Бот пришлет какую-то информацию и ваш **токен** - сохраняем его, и вставляем ниже



In [ ]:
import telebot

API_TOKEN = ... # Вот сюда надо поставить ваш токен в формате 'буквабуквацифрабуква'

MAX_LEN = 75

last_generated = ''

bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Отправляй мне начало историй, а я буду их продолжать :)')
    bot.send_message(message.chat.id, 'Сейчас максимальная длина рассказа - ' + str(MAX_LEN) + ' букв, но ты можешь сделать ее больше командами!')

@bot.message_handler(commands=['increase_len'])
def increase_len(message):
  global MAX_LEN
  if MAX_LEN < 200:
    MAX_LEN += 25
    bot.send_message(message.chat.id, 'Увеличил максимальное количество букв до {}'.format(MAX_LEN))
  else:
    bot.send_message(message.chat.id, 'Уже и так много :(')

@bot.message_handler(commands=['decrease_len'])
def decrease_len(message):
  global MAX_LEN
  if MAX_LEN > 25:
    MAX_LEN -= 25
    bot.send_message(message.chat.id, 'Уменьшил максимальное количество букв до {}'.format(MAX_LEN))
  else:
    bot.send_message(message.chat.id, 'Куда меньше то? :(')

@bot.message_handler(commands=['continue'])
def continue_text(message):
  global last_generated
  generated_story = generate_text(last_generated, len(last_generated) + MAX_LEN).split('<s>')[0]
  last_generated = generated_story 

  bot.send_message(message.chat.id, generated_story)

@bot.message_handler(content_types=['text'])
def generate_story(message):
  global last_generated
  generated_story = generate_text(message.text, min(MAX_LEN, len(message.text))).split('<s>')[0]
  last_generated = generated_story

  bot.send_message(message.chat.id, generated_story)

while True:
    bot.polling(none_stop=True)